In [5]:
import cv2
import numpy as np

SPICE_PARAMETERS = {
    "chili": {
        "aspect_ratio": (0.2, 5.0),
        "min_area": 5000,
        "color_range": ([0, 35, 55], [2, 255, 255]),
    },
    "pepper": {
        "aspect_ratio": (0.8, 1.2),
        "min_area": 300,
        "color_range": ([0, 0, 20], [180, 100, 80]),
    },
    "cardamom": {
        "aspect_ratio": (0.5, 3.0),
        "min_area": 500,
        "color_range": ([25, 40, 30], [50, 255, 255]),
    },
    "turmeric": {
        "aspect_ratio": (0.2, 3.0),
        "min_area": 2000,
        "color_range": ([17, 195, 75], [30, 255, 255]),
    },
    "clove": {
        "aspect_ratio": (0.2, 2.5),
        "min_area": 500,
        "color_range": ([0, 0, 10], [10, 180, 85]),
    },
}

def apply_edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    return cv2.dilate(edges, kernel, iterations=1)

def clean_mask(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    return cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

def compute_similarity(contour, mask):
    mask_contour = np.zeros_like(mask)
    cv2.drawContours(mask_contour, [contour], -1, 255, thickness=cv2.FILLED)
    mask_inside = cv2.bitwise_and(mask, mask_contour)
    overlap = np.count_nonzero(mask_inside)
    total_area = cv2.contourArea(contour)
    return overlap / total_area if total_area > 0 else 0

def draw_text_with_background(image, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, text_color=(255, 255, 255), bg_color=(0, 0, 0), thickness=2):
    """Draws text with a solid background to improve visibility."""
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    text_w, text_h = text_size
    x, y = position
    cv2.rectangle(image, (x, y - text_h - 5), (x + text_w + 5, y + 5), bg_color, -1)
    cv2.putText(image, text, (x, y), font, font_scale, text_color, thickness)

def extract_and_classify_contours(image, color_masks, edge_mask, spice_params):
    contours, _ = cv2.findContours(edge_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w * h < 100:
            continue

        best_spice = None
        best_similarity = 0

        for spice, mask in color_masks.items():
            similarity = compute_similarity(contour, mask)
            if similarity > best_similarity:
                best_similarity = similarity
                best_spice = spice

        if best_spice and best_similarity > 0.3:
            params = spice_params[best_spice]
            aspect_ratio = float(w) / h

            if params["aspect_ratio"][0] <= aspect_ratio <= params["aspect_ratio"][1]:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 3)

                # Ensure label is placed inside the bounding box
                text_x, text_y = x + 10, y + 30
                draw_text_with_background(image, best_spice.capitalize(), (text_x, text_y), font_scale=3, thickness=5)

    return image

def test_all_spices(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found.")
        return

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    edge_mask = apply_edge_detection(image)

    color_masks = {}

    for spice, params in SPICE_PARAMETERS.items():
        lower_color = np.array(params["color_range"][0])
        upper_color = np.array(params["color_range"][1])
        mask = cv2.inRange(hsv, lower_color, upper_color)
        mask = clean_mask(mask)
        color_masks[spice] = mask

        cv2.imshow(f"Mask - {spice.capitalize()}", cv2.resize(mask, (400, 300)))

    result_image = extract_and_classify_contours(image.copy(), color_masks, edge_mask, SPICE_PARAMETERS)

    cv2.imshow("Final Classification", cv2.resize(result_image, (800, 600)))

    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Path to the input image
image_path = r"C:\Users\vidya\Downloads\IMG_20250216_202121_2(1).jpg"
test_all_spices(image_path)


In [8]:
import cv2
import numpy as np
import imutils
from sklearn.cluster import KMeans

# Constants for spice detection
SPICE_PARAMETERS = {
    "chili": {
        "aspect_ratio": (0.2, 5.0),
        "min_area": 5000,
        "color_range": ([0, 35, 55], [2, 255, 255]),
    },
    "pepper": {
        "aspect_ratio": (0.8, 1.2),
        "min_area": 300,
        "color_range": ([0, 0, 20], [180, 100, 80]),
    },
    "cardamom": {
        "aspect_ratio": (0.5, 3.0),
        "min_area": 500,
        "color_range": ([25, 40, 30], [50, 255, 255]),  # Green range for cardamom
        "quality": {
            "bad_hue_min": 25,
            "bad_hue_max": 35,
            "bad_sat_min": 50,
            "bad_val_min": 50,
            "crack_val_max": 50,
            "crack_ratio_threshold": 0.01,
        },
    },
    "turmeric": {
        "aspect_ratio": (0.2, 3.0),
        "min_area": 2000,
        "color_range": ([17, 195, 75], [30, 255, 255]),
    },
    "clove": {
        "aspect_ratio": (0.2, 2.5),
        "min_area": 500,
        "color_range": ([0, 0, 10], [10, 180, 85]),
    },
}

resize_scale = 0.5

def resize_image(image, scale):
    width = int(image.shape[1] * scale)
    height = int(image.shape[0] * scale)
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

def apply_edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    return cv2.dilate(edges, kernel, iterations=1)

def clean_mask(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    return cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

def compute_similarity(contour, mask):
    mask_contour = np.zeros_like(mask)
    cv2.drawContours(mask_contour, [contour], -1, 255, thickness=cv2.FILLED)
    mask_inside = cv2.bitwise_and(mask, mask_contour)
    overlap = np.count_nonzero(mask_inside)
    total_area = cv2.contourArea(contour)
    return overlap / total_area if total_area > 0 else 0

def draw_text_with_background(image, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=3, text_color=(255, 255, 255), bg_color=(0, 0, 0), thickness=3):
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    text_w, text_h = text_size
    x, y = position
    cv2.rectangle(image, (x, y - text_h - 5), (x + text_w + 5, y + 5), bg_color, -1)
    cv2.putText(image, text, (x, y), font, font_scale, text_color, thickness)

def extract_and_classify_contours(image, color_masks, edge_mask, spice_params):
    contours, _ = cv2.findContours(edge_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w * h < 100:  # Minimum size filter
            continue

        best_spice = None
        best_similarity = 0

        for spice, mask in color_masks.items():
            similarity = compute_similarity(contour, mask)
            if similarity > best_similarity:
                best_similarity = similarity
                best_spice = spice

        if best_spice and best_similarity > 0.3:
            params = spice_params[best_spice]
            aspect_ratio = float(w) / h

            if params["aspect_ratio"][0] <= aspect_ratio <= params["aspect_ratio"][1]:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 3)

                # Basic spice label
                text_x, text_y = x + 10, y + 30
                draw_text_with_background(image, best_spice.capitalize(), (text_x, text_y), font_scale=3, thickness=3)

                # Additional quality check for cardamom
                if best_spice == "cardamom":
                    roi = image[y:y+h, x:x+w]
                    roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                    single_cardamom_mask = np.zeros_like(roi_hsv[:, :, 0])
                    cv2.drawContours(single_cardamom_mask, [contour], -1, 255, thickness=cv2.FILLED)
                    mean_hsv = cv2.mean(roi_hsv, mask=single_cardamom_mask)[:3]
                    hue, sat, val = mean_hsv

                    # Crack detection
                    crack_mask = cv2.inRange(roi_hsv, (0, 0, 0), (180, 255, params["quality"]["crack_val_max"]))
                    crack_mask = cv2.bitwise_and(crack_mask, single_cardamom_mask)
                    kernel_dilate = np.ones((3, 3), np.uint8)
                    crack_mask = cv2.dilate(crack_mask, kernel_dilate, iterations=1)
                    crack_pixels = cv2.countNonZero(crack_mask)
                    total_pixels = cv2.countNonZero(single_cardamom_mask)
                    crack_ratio = crack_pixels / total_pixels if total_pixels > 0 else 0

                    # Quality classification
                    is_brown_yellow = (params["quality"]["bad_hue_min"] <= hue <= params["quality"]["bad_hue_max"] and
                                     sat >= params["quality"]["bad_sat_min"] and
                                     val >= params["quality"]["bad_val_min"])
                    has_cracks = crack_ratio > params["quality"]["crack_ratio_threshold"]
                    quality = "Bad" if is_brown_yellow or has_cracks else "Good"

                    # Add quality label below spice name
                    quality_text_y = y + 60
                    draw_text_with_background(image, f"Quality: {quality}", (text_x, quality_text_y), font_scale=3, thickness=3)

                    print(f"Cardamom {x},{y} - HSV: H:{hue:.1f} S:{sat:.1f} V:{val:.1f}")
                    print(f"Cardamom {x},{y} - Crack Ratio: {crack_ratio:.2f}")
                    print(f"Cardamom {x},{y} - Brown/Yellow: {is_brown_yellow}, Has Cracks: {has_cracks}")
                    print(f"Cardamom {x},{y} - Labeled as: {quality}")

    return image

def test_all_spices(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found.")
        return

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    edge_mask = apply_edge_detection(image)

    color_masks = {}
    for spice, params in SPICE_PARAMETERS.items():
        lower_color = np.array(params["color_range"][0])
        upper_color = np.array(params["color_range"][1])
        mask = cv2.inRange(hsv, lower_color, upper_color)
        mask = clean_mask(mask)
        color_masks[spice] = mask
        cv2.imshow(f"Mask - {spice.capitalize()}", cv2.resize(mask, (400, 300)))

    result_image = extract_and_classify_contours(image.copy(), color_masks, edge_mask, SPICE_PARAMETERS)

    cv2.imshow("Final Classification", cv2.resize(result_image, (800, 600)))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Path to the input image
image_path = r"C:\Users\vidya\Downloads\IMG_20250216_202121_2(1).jpg"
test_all_spices(image_path)

Cardamom 515,1658 - HSV: H:0.0 S:0.0 V:0.0
Cardamom 515,1658 - Crack Ratio: 0.00
Cardamom 515,1658 - Brown/Yellow: False, Has Cracks: False
Cardamom 515,1658 - Labeled as: Good
Cardamom 2959,1014 - HSV: H:0.0 S:0.0 V:0.0
Cardamom 2959,1014 - Crack Ratio: 0.00
Cardamom 2959,1014 - Brown/Yellow: False, Has Cracks: False
Cardamom 2959,1014 - Labeled as: Good


In [2]:
import cv2
import numpy as np

def resize_image(image, width, height):
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

def debug_hsv_ranges(image_path, spice):
    def nothing(x):
        pass

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Input image not found.")
        return

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    cv2.namedWindow(f"Adjust HSV for {spice}")

    cv2.createTrackbar("Lower H", f"Adjust HSV for {spice}", 0, 180, nothing)
    cv2.createTrackbar("Lower S", f"Adjust HSV for {spice}", 0, 255, nothing)
    cv2.createTrackbar("Lower V", f"Adjust HSV for {spice}", 10, 255, nothing)
    cv2.createTrackbar("Upper H", f"Adjust HSV for {spice}", 180, 180, nothing)
    cv2.createTrackbar("Upper S", f"Adjust HSV for {spice}", 100, 255, nothing)
    cv2.createTrackbar("Upper V", f"Adjust HSV for {spice}", 80, 255, nothing)

    while True:
        lower_h = cv2.getTrackbarPos("Lower H", f"Adjust HSV for {spice}")
        lower_s = cv2.getTrackbarPos("Lower S", f"Adjust HSV for {spice}")
        lower_v = cv2.getTrackbarPos("Lower V", f"Adjust HSV for {spice}")
        upper_h = cv2.getTrackbarPos("Upper H", f"Adjust HSV for {spice}")
        upper_s = cv2.getTrackbarPos("Upper S", f"Adjust HSV for {spice}")
        upper_v = cv2.getTrackbarPos("Upper V", f"Adjust HSV for {spice}")

        lower_color = np.array([lower_h, lower_s, lower_v])
        upper_color = np.array([upper_h, upper_s, upper_v])

        mask = cv2.inRange(hsv, lower_color, upper_color)
        result = cv2.bitwise_and(image, image, mask=mask)

        cv2.imshow(f"Original {spice}", resize_image(image, 800, 600))
        cv2.imshow(f"Mask for {spice}", resize_image(mask, 800, 600))
        cv2.imshow(f"Result for {spice}", resize_image(result, 800, 600))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    print(f"Final HSV Range for {spice}: Lower = {lower_color.tolist()}, Upper = {upper_color.tolist()}")
    cv2.destroyAllWindows()

debug_hsv_ranges(
    image_path=r"C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Approch-2\Cardamom\IMG_20241102_130553_14.jpg",
    spice="pepper"
)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2561: error: (-27:Null pointer) NULL window: 'Adjust HSV for pepper' in function 'cvGetTrackbarPos'
